Ref: 

http://gaussianprocess.org/gpml/data/

Dataset is taken from: https://www.openml.org/search?type=data&sort=runs&id=44148&status=active

In [9]:
import sys
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

sys.path.append("../")

In [10]:
print(tf.__version__)

2.14.1


These features are excluded according to openml: 
number_of_elements, range_Valence

In [11]:
filepath = 'superconduct.csv'

# read dataset
df = pd.read_csv(filepath)
#df = df.drop(columns=['number_of_elements','range_Valence'])
columns = list(df.columns)
target = 'critical_temp'
print(df.shape)

(21263, 80)


In [8]:
from sklearn.model_selection import train_test_split

y = df[target].values
X = df.drop(columns=[target]).values

X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, train_size=13378, random_state=0)
X_train, X_vald, y_train, y_vald = train_test_split(X_tmp, y_tmp, train_size=10000, random_state=0)

print(f'X_train.shape: {X_train.shape}')
print(f'X_vald.shape: {X_vald.shape}')
print(f'X_test.shape: {X_test.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'y_vald.shape: {y_vald.shape}')
print(f'y_test.shape: {y_test.shape}')

X_train.shape: (10000, 79)
X_vald.shape: (3378, 79)
X_test.shape: (7885, 79)
y_train.shape: (10000,)
y_vald.shape: (3378,)
y_test.shape: (7885,)


In [14]:
len([target])

1

In [36]:
# Normalize target column
'''
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_train = scaler.fit_transform(y_train.reshape((-1,1)))
y_vald = scaler.transform(y_vald.reshape((-1,1)))
y_test = scaler.transform(y_test.reshape((-1,1)))

y_train = y_train.ravel()
y_vald = y_vald.ravel()
y_test = y_test.ravel()'''

In [10]:
def array_to_dataset(data, target, shuffle=True, batch_size=128):
    ds = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        ds = ds.shuffle(batch_size*2).batch(batch_size).prefetch(batch_size)
    else:
        ds = ds.batch(batch_size)
    return ds

batch_size = 512
train_ds = array_to_dataset(X_train, y_train, batch_size=batch_size)
vald_ds = array_to_dataset(X_vald, y_vald, shuffle=False, batch_size=batch_size)
test_ds = array_to_dataset(X_test, y_test, shuffle=False, batch_size=batch_size)

In [14]:
from ife import IFENetRegressor

n_features = X_train.shape[1]
_, counts = np.unique(y_train, return_counts=True)
n_response = len(target)
ife_num_layers = 1
clf_num_layers = 1
clf_hidden_units = [64]
reduction_layer = 'average'
num_att = 64
r = 4.15

print(f'n_response: {n_response}')
print(f'n_features: {n_features}')

ife_params = {'n_features': n_features,
              'n_outputs': n_response,
              'num_att': num_att,
              'r': r,
              'ife_num_layers': ife_num_layers, 
              'clf_num_layers': clf_num_layers,
              'clf_hidden_units': clf_hidden_units,
              'reduction_layer': reduction_layer
             }
model = IFENetRegressor(**ife_params)

n_response: 13
n_features: 81


In [15]:
loss_fn = tf.keras.losses.MeanSquaredError()

lr = 0.015
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, 
                                                              decay_steps=2000,
                                                              decay_rate=0.95,
                                                              staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

checkpoint_path = 'checkpoints/ifeNet_superconduct.h5'
patience = 100
callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss'),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, monitor='val_loss')]

epochs = 100
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['mae'])

In [ ]:
saved_model_path = 'saved_model/ifeNet_superconduct.h5'
model.fit(train_ds, validation_data=vald_ds, epochs=epochs, callbacks=callbacks, verbose=2)
model.load_weights(checkpoint_path)
model.save_weights(saved_model_path)

Epoch 1/100


In [16]:
y_pred = np.empty((0,))
y_test = np.empty((0,))

for data,label in test_ds:
    y_hat = model(data)
    y_pred = np.append(y_pred, y_hat.numpy().ravel())

    label = label.numpy()
    y_test = np.append(y_test, label.ravel())

In [24]:
y_pred.max()

0.6583020687103271

In [25]:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

0.4528119753756158
0.0054882658180464205
0.056801059710301555
